# Obstacle Avoidance Simulation

### THE FOLLOWING EXERCISE WAS GIVEN AS A PART OF THE COURSE IN THE CONSTRUCT AI

Write a Publisher and Subscriber node:

Create a new ROS2 package named robot_patrol and add rclpy, std_msgs, sensor_msgs, and geometry_msgs as dependencies.
Use the previous publisher to create a new code that makes the robot avoid obstacles while continuously moving around the office. This behavior is kwown as patrol behavior.
Get the real-time data from the laser and decide the robot's movement based on it. An example logic would be the following:

If the distance to the detected obstacle is bigger than the distance thresholds, just keep the robot moving forward
If the distance to the detected obstacle is smaller than a certain threshold (ie. 0.15m) in either side (right or left) of the robot, you need to make the robot turn to the opposite direction in order to move away from it
If the distance to the detected obstacle is smaller than a certain threshold (ie. 0.55m) in front of the robot, you need to rotate the robot to the direction (right or left) which is more clear of obstacles.

Hint: Use the following ranges:-

right_side = msg.ranges[2246]
\
front = msg.ranges[0]
\
left_side = msg.ranges[748]

Create the launch file and test your program!

### 1. Create a workspace called "ros2_ws"

In [ ]:
# SHELL:

mkdir -p ~/ros2_ws/src

### 2. Create a package called as "ob_avoid_pkg"

In [ ]:
# SHELL:

cd src # move to the 'src' folder

ros2 pkg create --build-type ament_python ob_avoid_pkg --dependencies rclpy std_msgs sensor_msgs geometry_msgs
# create a new package

### 3. Create a '.py' file
Create a file named as 'code.py' in the 'ob_avoid_pkg' folder next to the 'init.py' file.

In [ ]:
# CODE TERMINAL
# code.py

#!/usr/bin/env python3

import rclpy
from rclpy.node import Node
from geometry_msgs.msg import TwistStamped
from sensor_msgs.msg import LaserScan

class Obavoid(Node):

    def __init__(self):
        super().__init__('obavoid_node')

        # Create a subscription to laser scan data
        self.scan_sub = self.create_subscription(
            LaserScan,
            'scan',
            self.laser_callback,
            10)

        # Create a publisher for publishing velocity commands
        self.cmd_pub = self.create_publisher(
            TwistStamped,
            '/rosbot_xl_base_controller/cmd_vel',
            5)

        # Create a timer for controlling the robot's movement
        self.control_timer = self.create_timer(
            0.5,  # 500 ms
            self.motion)

        # Initialize the velocity command message
        self.cmd = TwistStamped()
        self.cmd.twist.linear.x = 0.0
        self.cmd.twist.angular.z = 0.0

        # Constants for distance thresholds
        self.min_distance = 0.55
        self.side_threshold = 0.15

        # Initialize variables for laser scan data
        self.left_side = 0.0
        self.front = 0.0
        self.right_side = 0.0


    def laser_callback(self, msg: LaserScan):
        # Process laser scan data and extract  values for different sections
        # of the robot's surroundings
        self.right_side = msg.ranges[2246]
        self.front = msg.ranges[0]
        self.left_side = msg.ranges[748]
        self.get_logger().info('I receive: "%s"' % str(self.front))


    def motion(self):
        # Control the robot's movement based on laser scan data

        linear_vel = 0.5

        # Determine robot's movement based on obstacle detection
        if self.left_side < self.side_threshold and self.front > self.min_distance:
            self.get_logger().warning('Reduce speed and turn!!')
            self.cmd.twist.linear.x = linear_vel * 0.5
            self.cmd.twist.angular.z = -0.15
        elif self.right_side < self.side_threshold and self.front > self.min_distance:
            self.get_logger().warning('Reduce speed and turn!!')
            self.cmd.twist.linear.x = linear_vel * 0.5
            self.cmd.twist.angular.z = 0.15
        elif self.front > self.min_distance:
            self.get_logger().info('Moving forward!!')
            self.cmd.twist.linear.x = linear_vel
            self.cmd.twist.angular.z = 0.0
        elif self.front < self.min_distance:
            self.get_logger().error('Stop and rotate!!')
            if self.compare_sides(self.left_side, self.right_side):
                self.cmd.twist.linear.x = linear_vel * 0.25
                self.cmd.twist.angular.z = 0.35
            else:
                self.cmd.twist.linear.x = linear_vel * 0.25
                self.cmd.twist.angular.z = -0.35

        # Publish velocity command
        self.cmd_pub.publish(self.cmd)

    def compare_sides(self, left, right):
        # Compare distances sensed on left and right sides
        return left >= right


def main(args=None):
    # Initialize ROS2 node
    rclpy.init(args=args)

    # Create an instance of the Patrol class
    obavoid_node = Obavoid()

    try:
        # Spin the node to handle callbacks
        rclpy.spin(obavoid_node)
    except KeyboardInterrupt:
        pass
    finally:
        # Shutdown ROS2 node
        obavoid_node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()



### 4. Create the Launch File in th Launch folder

In [ ]:
# SHELL:

cd ~/ros2_ws/src/ob_avoid_pkg
mkdir launch

cd ~/ros2_ws/src/ob_avoid_pkg/launch
touch launch_node.launch.py


### 5. Fillup the launch file

In [ ]:
# CODE TERMINAL
# launch_node.launch.py

from launch import LaunchDescription
from launch_ros.actions import Node

def generate_launch_description():
    return LaunchDescription([
        Node(
            package='ob_avoid_pkg',
            executable='exe',
            output='screen',
            emulate_tty=True
        ),
    ])

### 6. Modify the setup

In [ ]:
# CODE TERMINAL
# setup.py

from setuptools import find_packages, setup
import os
from glob import glob

package_name = 'ob_avoid_pkg'

setup(
    name=package_name,
    version='0.0.0',
    packages=find_packages(exclude=['test']),
    data_files=[
        ('share/ament_index/resource_index/packages',
            ['resource/' + package_name]),
        ('share/' + package_name, ['package.xml']),
        (os.path.join('share', package_name), glob('launch/*.launch.py'))
    ],
    install_requires=['setuptools'],
    zip_safe=True,
    maintainer='user',
    maintainer_email='user@todo.todo',
    description='TODO: Package description',
    license='TODO: License declaration',
    tests_require=['pytest'],
    entry_points={
        'console_scripts': [
            'exe = ob_avoid_pkg.code:main'
        ],
    },
)

### 7. Compile the  package

In [ ]:
cd ~/ros2_ws
colcon build --packages-select ob_avoid_pkg
source ~/ros2_ws/install/setup.bash

### 8. launch the package

In [ ]:
ros2 launch robot_patrol start_patrolling.launch.py

# Result

#### we get the following result in the simulation window :

*   Robot proceeds forward if the detected obstacle is more than 0.55 meters in front of the robot.


*   The robot should turn in the opposite direction in order to move away from the detected obstacle if it is less than 0.15 meters from either the robot's right or left side.

*   The robot spins to the left or right, whatever is more obstacle-free, if the identified obstruction is less than 0.55 meters in front of the robot.




